In [ ]:
import base64
import requests
from io import BytesIO
from PIL import Image
import os
from dotenv import load_dotenv

# load_dotenv()
api_key = os.getenv("HYPERBOLIC_API_KEY")

def encode_image(img):
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    encoded_string = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return encoded_string


img = Image.open("chatbot/backend/vlm/image.png")
base64_img = encode_image(img)

api = "https://api.hyperbolic.xyz/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}",
}

payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What is this image?"},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_img}"},
                },
            ],
        }
    ],
    "model": "Qwen/Qwen2-VL-7B-Instruct",
    "max_tokens": 2048,
    "temperature": 0.7,
    "top_p": 0.9,
}

response = requests.post(api, headers=headers, json=payload)
print(response.json())

{'id': 'chat-439ee6fcae9749dab70476cd5065010e', 'object': 'chat.completion', 'created': 1738738620, 'model': 'Qwen/Qwen2-VL-7B-Instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'This image is an architecture diagram for a Read-Ask-Generate (RAG) system. The diagram is divided into four main sections: RAG Workflow, Deployment and Scaling, Analytics & Dashboard, and a Legend. \n\n1. **RAG Workflow**:\n   - **User** submits a query.\n   - The query is processed by the **LLM (Large Language Model)**, which retrieves the top K most similar chunks/docs from the **Milvus Vector Database**.\n   - The retrieved documents are passed to the **Firebase Relational Database** for storing user chat history.\n   - The LLM generates an answer, which is sent to the **User**.\n\n2. **Deployment and Scaling**:\n   - Users submit queries through a **Query Message Queue**.\n   - The queries are distributed to multiple **RAG Servers** (Server 1, Server 2, Server 3) through a **L